In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Feel free to change directory on your owns
# GPU is needed for this notebook

%cd /content/drive/MyDrive/Colab Notebooks/NLP/argmine/argmining-21-keypoint-analysis-sharedtask-code/code/src-ipynb

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/NLP/argmine/argmining-21-keypoint-analysis-sharedtask-code/code/src-ipynb


In [2]:
!pip install fast-pagerank;
!pip install py-rouge;
!pip install sentence_transformers;

     |████████████████████████████████| 56 kB 2.2 MB/s 
     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 3.5 MB 11.2 MB/s 
     |████████████████████████████████| 6.8 MB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 596 kB 48.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=75f116d03f55f66a0c3cf6f9bb89345701afec1c44d5bd54fc026ca446a3be1c
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import rouge
import os

In [4]:
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [5]:
import json
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import nltk
nltk.download('punkt')

from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power


pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [7]:
train_args = pd.read_pickle('../../data/scores/train_args_with_scores.pkl')

In [8]:
train_topic_with_scored_kps = pd.read_pickle('../../data/scores/train_topic_keypoints_with_scores.pkl')

In [9]:
train_topics = train_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items]),
    'gt-kps' : lambda x: set([item for items in x for item in items]),
    #'gt-cand-kps': lambda x: set([item for items in x for item in items])
}).reset_index()

In [10]:
train_topics.columns

Index(['topic', 'stance', 'sents_with_scores', 'gt-kps'], dtype='object')

In [11]:
train_topics.head()

topic  ...                                                                                                                                                                                                                                                                                                                                                                                                                   gt-kps
0  Assisted suicide should be a criminal offence  ...                                                                                                                                                                                           {People should have the freedom to choose to end their life, Assisted suicide reduces suffering, The terminally ill would benefit from assisted suicide, Assisted suicide gives dignity to the person that wants to commit it}
1  Assisted suicide should be a criminal offence  ...                          {Assisted suicide allows people to solicit someone to die to their own benefit, Assisted suicide should not be allowed because many times people can still get better, Assisted suicide is akin to killing someone, People who are ill do not have clear judgement and are in need of help, Assisted suicide violates the doctor's role, People do not have the right to self-harm/harm others}
2                 Homeschooling should be banned  ...  {Homeschooling is often the best option for catering for the needs of exceptional/religious/ill/disabled students., Homeschools can be personalized to the child's pace/needs, Parents should be permitted to choose the education of their children, Mainstream schools have a lot of violence/bullying, Parents will have more ability to pay-attention/educate their child, The home is a good learning environment}
3                 Homeschooling should be banned  ...                                                                                                                                                                                                                   {Parents are not qualified as teachers, Mainstream schools are of higher educational quality, Homeschools cannot be regulated/standardized, Mainstream schools are essential to develop social skills}
4        The vow of celibacy should be abandoned  ...                                                                                                                                 {Removing celibacy could lead to sexually reckless acts, Religious experiences and traditions should be maintained, Celibacy allows a priest to focus on God, Celibacy makes the priests to be perceived as holy, Celibacy is a strong religious belief, Priests choose to be celibates}

[5 rows x 4 columns]

In [12]:
#Add to the candidate sentences from the topic, the topic key-points
train_topics['sents_with_scores'] = train_topics.apply(lambda row: list(row['sents_with_scores']) + train_topic_with_scored_kps[(train_topic_with_scored_kps.topic == row['topic']) & (train_topic_with_scored_kps.stance == row['stance'])]['scored_kps'].tolist()[0], axis=1)

In [13]:
model = SentenceTransformer('../../data/siamese-models/roberta-base-contrastive-10-epochs-2022-02-03_13-49-59')

In [14]:
def gen_match_matrix(model, sents, topic, min_match_score=0):
    sents1 = [topic + ' <SEP> ' + x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def evalute_topic_by_rouge(row):
    ranked_sents = [x[0] for x in row['ranked_sents']][0:10]
    # gt_kps = [[x] for x in row['gt-kps']]
    gt_kps = [x for x in row['gt-kps']]

    r_l = []
    for a,b in itertools.product(gt_kps, ranked_sents):
        # print(a,b)
        # print(type(a), type(b))
        scores = evaluator.get_scores(a, b)
        r_l.append(scores['rouge-l']['f'])
    
    return np.mean(r_l)

def evalute_topic(row):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    # ranked_sents = [x for x in row['filtered_sents']]
    gt_kps_ranks = []
    for real_kp in row['gt-kps']:
        if real_kp in ranked_sents:
            gt_kps_ranks.append(ranked_sents.index(real_kp)+1)
        else:
            print('kp is not there...')
            gt_kps_ranks.append(len(ranked_sents)+1)
            
    #print(gt_kps_ranks)
    return 1/(sum(gt_kps_ranks)/len(gt_kps_ranks)) if len(gt_kps_ranks) > 0 else 0

In [15]:
def apply_page_rank(row, p=0.85, min_quality_score=0.5, min_match_score=0.5, min_len=5, max_len=20):
    cand_sents = [x for x in row['sents_with_scores'] if x[1] > min_quality_score and len(x[0].split()) < max_len and len(x[0].split()) > min_len]
    if len(cand_sents) == 0:
        return []
        print('empty')
    cands, cands_qualities = zip(*cand_sents)
    #print(cands)
    #print(cands_qualities)
    cands_matching_mat = gen_match_matrix(model, cands, row['topic'], min_match_score=min_match_score)
    pr=pagerank(cands_matching_mat, personalize=np.array(cands_qualities), p=p) #it looks like modifying the initial probability doesn't help
    #pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cands, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])    

In [16]:
def filter_ranked_list(row, model, min_match=0.8):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    filtered_sents = []
    while len(filtered_sents) < 5:
        filtered_sents = []
        for i, s in enumerate(ranked_sents):
            if len(filtered_sents) == 0:
                filtered_sents.append(s)
                print('FFFF ' , s)
            else:
                matching_scores = gen_match_matrix(model, [s]+filtered_sents, row['topic'])
                max_sim = np.max(matching_scores[0][1:])
                #print(matching_scores)
                if max_sim < min_match:
                    print('SSSS ' , s)
                    filtered_sents.append(s)
        
        min_match = min_match + 0.1
        print(min_match)

    return filtered_sents

In [17]:
scores = []
for min_match_score in [0.2, 0.4, 0.6, 0.8, 1.0]:
    for p in [0.2, 0.4, 0.8, 1.0]:
        for min_quality_score in [0.2, 0.5, 0.8]:
            train_topics['ranked_sents'] = train_topics.apply(lambda row: apply_page_rank(row, p=p, min_quality_score=min_quality_score, min_match_score=min_match_score), axis=1)
            train_topics['scores'] = train_topics.apply(lambda row: evalute_topic_by_rouge(row), axis=1)
            score = np.mean(train_topics.scores.tolist())
            scores.append([p, min_quality_score, min_match_score, score])

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/

In [18]:
scores_df = pd.DataFrame(scores, columns=['p', 'min-quality-score', 'min-match-score', 'score'])
scores_df.sort_values('score', ascending=False)

,p,min-quality-score,min-match-score,score
36,0.2,0.2,0.8,0.251834
37,0.2,0.5,0.8,0.251830
38,0.2,0.8,0.8,0.251378
48,0.2,0.2,1.0,0.249958
56,0.8,0.8,1.0,0.249958
49,0.2,0.5,1.0,0.249958
54,0.8,0.2,1.0,0.249958
53,0.4,0.8,1.0,0.249958
55,0.8,0.5,1.0,0.249958
52,0.4,0.5,1.0,0.249958


-------------

##### Apply on test:

In [19]:
# test_kps = pd.read_csv('../../data/key_points_our_test.csv')

test_kps = pd.read_csv('../../data/test_data/key_points_test.csv')

In [20]:
test_args = pd.read_pickle('../../data/scores/test_args_with_scores.pkl')

In [21]:
test_args.head()

,arg_id,argument,topic,stance,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory since children don't spread the virus],"[(Routine child vaccinations isn't mandatory since children don't spread the virus, 0.714194655418396)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be mandatory because children may not bear the side effects of it.],"[(Routine child vaccinations should not be mandatory because children may not bear the side effects of it., 0.8710675239562988)]"
2,arg_0_2,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be necessary as children can't catch the disease],"[(Routine child vaccinations should not be necessary as children can't catch the disease, 0.8207754492759705)]"
3,arg_0_3,A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children,Routine child vaccinations should be mandatory,-1,[A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children],"[(A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children, 0.8951572775840759)]"
4,arg_0_4,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,"[As long as vaccines are not free of side effects, it cannot make them mandatory for our children.]","[(As long as vaccines are not free of side effects, it cannot make them mandatory for our children., 0.8620932698249817)]"


In [22]:
test_topics = test_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items])
}).reset_index()

In [23]:
test_topics['ranked_sents'] = test_topics.apply(lambda row: apply_page_rank(row, p=0.2, min_quality_score=0.8, min_match_score=0.8), axis=1)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:138: SparseE

In [24]:
test_topics['ranked_kps'] = test_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.8), axis=1)

FFFF  child vaccinations shouldn't be mandatory because the vaccine can have side effects on children
SSSS  parents should have the freedom to decide what they consider best for their children
SSSS  The child population has a low degree of vulnerability, so vaccination is not urgent yet
SSSS  When vaccines are mandatory, they can infringe on family religious choices
SSSS  Linking a measure as good as vaccination to coercive measures would cause serious harm
SSSS  Making child vaccination compulsory is not the the best way to obtain high vaccination rates
SSSS  vaccination of children should be limited as it affects child development
SSSS  child vaccinations is not mandatory because it may cost the country unnecessary funds.
SSSS  No-one can tell a parent that they must vaccinate their child, it is against their human rights.
SSSS  Vaccines contain heavy metals, they are toxic to children.
SSSS  the poorest people may not be able to pay for the vaccine
SSSS  childcare is the responsibil

In [25]:
#adding gt-kps
test_topics['gt-kps']= test_topics.apply(lambda row: test_kps[(test_kps.topic==row['topic']) & (test_kps.stance==row['stance'])]['key_point'].tolist(), axis=1)

In [26]:
test_topics.head()

topic  ...                                                                                                                                                                                                                                                                                                              gt-kps
0                Routine child vaccinations should be mandatory  ...                                                                              [Routine child vaccinations, or their side effects, are dangerous, Mandatory vaccination contradicts basic rights, The parents and not the state should decide, Routine child vaccinations are not necessary to keep children healthy]
1                Routine child vaccinations should be mandatory  ...                                      [Routine child vaccinations are effective, Child vaccination saves lives, Routine child vaccinations are necessary to protect others, Routine child vaccinations should be mandatory to prevent virus/disease spreading, Children should not suffer from preventable diseases]
2  Social media platforms should be regulated by the government  ...      [Social media regulation is not effective, Social media regulation harms privacy, Social media regulation harm freedom of speech and other democratic rights, The government should not intervene in the affairs of a private company, Social media regulation can lead to political abuses by the government]
3  Social media platforms should be regulated by the government  ...  [Social media regulation protects the younger audiences, Social media regulation is necessary to deal with fake news, Social media regulation is beneficial to society at large, Social media regulation can help to deal with negative content, Social media regulation is required to deal with malicious users]
4                          The USA is a good country to live in  ...                                                   [The US has unfair health and education policies, The US has a problematic/divisive political system, The US has high taxation/high costs of living, The US is xenophobic/racist, The US has inequality/poverty, The US is unsafe, The US has a negative culture]

[5 rows x 6 columns]

In [27]:
test_topics[['topic', 'stance', 'ranked_kps', 'gt-kps']].to_pickle('../../data/pagerank-test-keypoints.pkl')

In [28]:
keypoints_ds = []
c = 0
for i, row in test_topics.iterrows():
    for kp in row['ranked_kps'][0:5]:
        keypoints_ds.append([c, row['topic'], row['stance'], kp])
        c+=1
        
keypoints_df = pd.DataFrame(keypoints_ds, columns=['key_point_id', 'topic', 'stance', 'key_point'])

In [29]:
keypoints_df.sample(n=5).head()

,key_point_id,topic,stance,key_point
26,26,The USA is a good country to live in,1,The USA has so many opportunities for everyone working their that it is a good country to live in.
7,7,Routine child vaccinations should be mandatory,1,child vaccinations should be mandatory because they are the future of the world and must be protected
27,27,The USA is a good country to live in,1,"The USA offers an abundance of vast open lands, wilderness and beautiful landscapes."
3,3,Routine child vaccinations should be mandatory,-1,"When vaccines are mandatory, they can infringe on family religious choices"
29,29,The USA is a good country to live in,1,The USA has a well established and stable political system


In [30]:
keypoints_df.to_csv('../../data/pagerank-generated-test-keypoints.csv', index=False)

In [31]:
keypoints_df.topic.value_counts()

Routine child vaccinations should be mandatory                  10
Social media platforms should be regulated by the government    10
The USA is a good country to live in                            10
Name: topic, dtype: int64